In [2]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import os
spark = SparkSession.builder.master("local").getOrCreate()
DATA_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", "data"))

In [4]:
users_path = f"{DATA_DIR}/parquet/users"
messages_path = f"{DATA_DIR}/parquet/messages"
users = spark.read.parquet(users_path)
messages = spark.read.parquet(messages_path)
users.show(5, False)
messages.show(5, False)

+---+---------+--------+----------+
|id |firstname|lastname|dob       |
+---+---------+--------+----------+
|1  |John     |Smith   |2001-01-01|
|2  |Kim      |Melly   |1998-08-28|
+---+---------+--------+----------+

+-------+-----------+-------------+----+---------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |
+-------+-----------+-------------+----+---------+----------+
|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|
|Title 2|Hello World|null         |3   |d@ch.com |2022-01-02|
+-------+-----------+-------------+----+---------+----------+



## Inner join

Records from both sides where the join key is found

In [5]:
messages.join(users, on=[messages.user == users.id], how="inner").show(5, False)

+-------+-----------+-------------+----+---------+----------+---+---------+--------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |id |firstname|lastname|dob       |
+-------+-----------+-------------+----+---------+----------+---+---------+--------+----------+
|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|2  |Kim      |Melly   |1998-08-28|
+-------+-----------+-------------+----+---------+----------+---+---------+--------+----------+



## Left join

Records from left side with data from right side *if the join key matches*, otherwise nulls from right.

In [6]:
messages.join(users, on=[messages.user == users.id], how="left").show(5, False)

+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |id  |firstname|lastname|dob       |
+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+
|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|2   |Kim      |Melly   |1998-08-28|
|Title 2|Hello World|null         |3   |d@ch.com |2022-01-02|null|null     |null    |null      |
+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+



## Right join

As the left join, but reversed.

In [7]:
# note messages and users have 'swapped sides' in the results and the code
users.join(messages, on=[messages.user == users.id], how="right").show(5, False)

+----+---------+--------+----------+-------+-----------+-------------+----+---------+----------+
|id  |firstname|lastname|dob       |m_title|m_body     |m_attachments|user|recipient|dt        |
+----+---------+--------+----------+-------+-----------+-------------+----+---------+----------+
|2   |Kim      |Melly   |1998-08-28|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|
|null|null     |null    |null      |Title 2|Hello World|null         |3   |d@ch.com |2022-01-02|
+----+---------+--------+----------+-------+-----------+-------------+----+---------+----------+



## Full Outer

Display data from either side, nulls where there is no match

In [8]:
messages.join(users, on=[messages.user == users.id], how="outer").show(5, False)

+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |id  |firstname|lastname|dob       |
+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+
|null   |null       |null         |null|null     |null      |1   |John     |Smith   |2001-01-01|
|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|2   |Kim      |Melly   |1998-08-28|
|Title 2|Hello World|null         |3   |d@ch.com |2022-01-02|null|null     |null    |null      |
+-------+-----------+-------------+----+---------+----------+----+---------+--------+----------+



## Semi

Rows from left side that match the right, but without the columns from the right.

In [9]:
messages.join(users, on=[messages.user == users.id], how="semi").show(5, False)

+-------+-----------+-------------+----+---------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |
+-------+-----------+-------------+----+---------+----------+
|Title  |Hello World|null         |2   |p@ch.com |2022-01-03|
+-------+-----------+-------------+----+---------+----------+



## Anti

Opposite of semi, rows from left that *don't match* the right, and just the left sides columns/data.

In [11]:
messages.join(users, on=[messages.user == users.id], how="anti").show(5, False)

+-------+-----------+-------------+----+---------+----------+
|m_title|m_body     |m_attachments|user|recipient|dt        |
+-------+-----------+-------------+----+---------+----------+
|Title 2|Hello World|null         |3   |d@ch.com |2022-01-02|
+-------+-----------+-------------+----+---------+----------+

